In [1]:
import numpy as np     
import nashpy as nash
import axelrod as axl
import time
import random

In [2]:
import main

In [13]:
game_ending_probs = np.linspace(0.001, 1-0.001, 10)
number_of_opponents = 1

while number_of_opponents <= 9:
    
    print(number_of_opponents)

    average_running_time_for_same_players = []
    for same_number_of_players_repeat in range(1, 6):
        
        print(same_number_of_players_repeat)

        players = main.who_is_playing(same_number_of_players)
        
        execution_time = []
        for same_opponents_repeat in range(5):

            print(same_opponents_repeat)
            print(players)

            initial_time = time.perf_counter()
            main.probabilities_of_defection(5, players, game_ending_probs, NE_alg, 123)
            final_time = time.perf_counter()
            execution_time.append(final_time - initial_time)
            
            print(execution_time)

        return execution_time

        print("Finished inner for loop!)

        mean_execution_time_for_same_players = sum(execution_time) / len(execution_time)
        average_running_time_for_same_players.append(mean_execution_time_for_same_players)

        print(average_running_time_for_same_players)

    return average_running_time_for_same_players
    
    print("Finished outer for loop!)

    mean_execution_time_for_same_num = sum(average_running_time_for_same_players) / len(average_running_time_for_same_players)
    average_running_time.append(mean_execution_time_for_same_num)

    print(average_running_time)

    num_of_opponents += 1

return(average_running_time)

1
2
3
4
5
6
7
8
9


In [1]:
algorithm_to_time_dict = {
    "Support Enumeration": [],
    "Vertex Enumeration": [],
    "Lemke Howson": []
}

In [2]:
for algorithm in algorithm_to_time_dict:
    algorithm_to_time_dict[algorithm].extend([3, 4, 5.99, 0.3])

In [3]:
algorithm_to_time_dict

{'Support Enumeration': [3, 4, 5.99, 0.3],
 'Vertex Enumeration': [3, 4, 5.99, 0.3],
 'Lemke Howson': [3, 4, 5.99, 0.3]}

In [5]:
import json

save_to_json = json.dumps(algorithm_to_time_dict)
json_file = open("trial_run.json", "w")
json_file.write(save_to_json)
json_file.close()